In [ ]:
import os
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, hamming_loss, f1_score

from skmultilearn.problem_transform import BinaryRelevance

# Baixar recursos do NLTK (necessário na primeira execução)
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

In [ ]:
# Carrega o arquivo de treino
try:
    df = pd.read_csv('data/train.csv')
except FileNotFoundError:
    df = pd.read_csv('data/train.csv/train.csv') # Alternativa caso a pasta seja criada de forma diferente

# Define as colunas de rótulos
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

print("Dimensões do DataFrame:", df.shape)
df.head()

In [ ]:
# Obtém a lista de stopwords em inglês
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    """
    Função para limpar e pré-processar o texto.
    """
    # Remove caracteres não alfanuméricos e converte para minúsculas
    text = re.sub(r'\W+', ' ', text).lower()
    
    # Tokeniza o texto
    tokens = word_tokenize(text)
    
    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    
    return " ".join(tokens)

# Aplica a função de pré-processamento na coluna de comentários
# Isso pode levar alguns minutos
print("Iniciando pré-processamento do texto...")
df['comment_processed'] = df['comment_text'].apply(preprocess_text)
print("Pré-processamento concluído.")

df[['comment_text', 'comment_processed']].head()

In [ ]:
# Define as features (X) e os rótulos (y)
X = df['comment_processed']
y = df[label_cols]

# Divide os dados em 80% para treino e 20% para teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Tamanho do conjunto de treino: {X_train.shape[0]} amostras")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]} amostras")

In [ ]:
# Inicializa o TfidfVectorizer
tfidf = TfidfVectorizer(max_features=5000, stop_words='english')

# Aprende o vocabulário e transforma os dados de treino
X_train_tfidf = tfidf.fit_transform(X_train)

# Apenas transforma os dados de teste com o vocabulário aprendido
X_test_tfidf = tfidf.transform(X_test)

print("Dimensões da matriz TF-IDF de treino:", X_train_tfidf.shape)
print("Dimensões da matriz TF-IDF de teste:", X_test_tfidf.shape)

In [ ]:
# Inicializa o classificador de Relevância Binária com Regressão Logística
# O parâmetro n_jobs=-1 utiliza todos os processadores disponíveis para acelerar
classifier = BinaryRelevance(classifier=LogisticRegression(solver='liblinear', random_state=42), require_dense=[False, True])

# Treina o classificador
print("Iniciando o treinamento do modelo...")
classifier.fit(X_train_tfidf, y_train)
print("Treinamento concluído.")

In [ ]:
# Faz previsões no conjunto de teste
predictions = classifier.predict(X_test_tfidf)

# Avalia o desempenho do modelo
accuracy = accuracy_score(y_test, predictions)
hamming = hamming_loss(y_test, predictions)
f1_micro = f1_score(y_test, predictions, average='micro')
f1_macro = f1_score(y_test, predictions, average='macro')

print("------ Desempenho do Classificador ------")
print(f"Pontuação de Acurácia (Subset Accuracy): {accuracy:.4f}")
print(f"Perda de Hamming (Hamming Loss): {hamming:.4f}")
print(f"Pontuação F1 (Micro): {f1_micro:.4f}")
print(f"Pontuação F1 (Macro): {f1_macro:.4f}")
print("---------------------------------------")

print("\nExplicação das Métricas:")
print(" - A Pontuação de Acurácia mede a proporção de amostras com TODOS os rótulos previstos corretamente. É uma métrica muito rigorosa.")
print(" - A Perda de Hamming é a fração de rótulos incorretamente previstos do total de rótulos.")
print(" - F1-Score (Micro) agrega as contribuições de todas as classes para calcular a média. É útil em casos de desbalanceamento de classes.")
print(" - F1-Score (Macro) calcula a métrica para cada rótulo e encontra sua média não ponderada. Dá peso igual a todas as classes.")